In [1]:
# Bin packing problem
# There are n items to be packed in U bins of size B.
# The sizes 0 ≤ si ≤ B of individual items are assumed to be known.
# The problem is to determine how to pack these n items in bins of size B,
# so that the number of required bins is minimum.
# xij indicates whether an item is packed in a given bin
# yj specifies if a bin is used in the solution or not

# a possible interger linear programming formulation of the problem
# min: sum(yj) for j in range(U)
# subject to:
#     sum(xij) = 1 for j in range(U) for i in range(n)
#     sum(si * xij) <= Byj for i in range(n) for j in range(U)
#     xij belongs to {0, 1} for i in range(n) and j in range(U)
#     yi belongs to {0, 1} for j in range(U)

In [2]:
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo

In [3]:
S = [5, 7, 5, 2, 4, 2, 5, 1, 6]
B = 10
n = U = len(S)

### The ILP formulation of the BPP


#### Using docplex


In [4]:
docplex_bp = Model(name="bin packing")

##### Define the decision variables


In [5]:
# binary variables x[i,j] indicate whether item i is packed in bin j
x = {
    (i, j): docplex_bp.binary_var(name="x_{0}_{1}".format(i, j))
    for i in range(n)
    for j in range(U)
}

# binary variables y[j] indicate whether bin j is used
y = {j: docplex_bp.binary_var(name="y_{0}".format(j)) for j in range(U)}

##### Set up the constraints


In [6]:
# each item must be in exactly one bin
for i in range(n):
    docplex_bp.add_constraint(docplex_bp.sum(x[i, j] for j in range(U)) == 1)

# each bin must be used for items that fit
for j in range(U):
    docplex_bp.add_constraint(
        docplex_bp.sum(S[i] * x[i, j] for i in range(n)) <= B * y[j]
    )

##### Express the objective


In [7]:
# minimize the number of bins used
docplex_bp.minimize(docplex_bp.sum(y[j] for j in range(U)))

##### Solve with the model


In [8]:
# result = docplex_bp.solve()
# docplex_bp.print_solution()

### Transform the ILP model into a QUBO


In [9]:
qp = from_docplex_mp(docplex_bp)
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)